# 🎬 Article-to-Video AI Toolkit

Welcome! This Colab notebook easily transforms text articles into engaging videos or audios using AI and Google Cloud tools (code from our [GitHub repo](https://github.com/google-marketing-solutions/article-to-video/tree/main)).

**Instructions:**

1.  **Setup:** Run the setup cells below to authenticate and configure your GCP Project ID & GCS Bucket. (Click ▶️ on cells, wait for ✅).
2.  **Upload:** Provide your article text file and corresponding images.
3.  **Generate:** Run the final cells to create your video/audio output, which will be available for download.

**Prerequisites:**

* Google Account
* Google Cloud Project ID
* Google Cloud Storage (GCS) Bucket (write permissions needed)
* Article text file (e.g., `.txt`) & Image files (e.g., `.jpg`, `.png`)
* Enabled Google Cloud APIs (incl. Text-to-Speech, Speech-to-Text, Storage, Language).

Let's get started!

# **1. Auth with Google Cloud and Set Up**

▶️ Remember to click the play button on the top left of each section once you've edited the default text so you can have it take effect. Run this code now to set up the Cloud Project.

In [1]:
#@markdown Insert your Google Cloud Project ID (this is the name of the project).
# This is a string, not an integer.
PROJECT_ID="article-to-video-test" # @param {type:"string"}
#@markdown insert your Google Cloud Storage Bucket that you will need for the output of the Text-to-Speech part of the solution. This will store the SRT (subtitles) and .wav (audio) files.
GCS_BUCKET="articletotextbucket" # @param {type:"string"}
# Language code string.
#@markdown * Choose the language the article is in. Defaults to US English.
#@markdown *      1.   en-US: US English
#@markdown *      2.   en-GB: British English
#@markdown *      3.   fr-FR: French (France)
#@markdown *      4.   de-DE: German (Germany)
#@markdown *      5.   es-ES: Spanish (Spain)
#@markdown *      6.   pt-BR: Portuguese (Brazil)
LANGUAGE_OF_ARTICLE = "en-GB" # @param ["en-US","en-GB","fr-FR","de-DE","es-ES","pt-BR"] {allow-input: false}

from google.colab import auth
auth.authenticate_user(project_id=PROJECT_ID)

!mkdir /content/input/

![Created Input Folder](https://raw.githubusercontent.com/google-marketing-solutions/article-to-video/main/colab/created_input_folder.png)

## **Upload Your Article and Images**

In [2]:
#@title ▶️ Upload Article File(s)
import os
from google.colab import files

input_dir = '/content/input/'
!mkdir -p {input_dir}

print("Upload ONE article file for single mode or multi-text:")
print("For multi-text, place one article after the next within same file.")
print("""For best results, label each article with a headline (e.g. 'Article 1')
"and make explicit its title & content (e.g. Title 1: 'XXX'. Content 1: XXX)""")
uploaded = files.upload()

# Use global scope so the variable is accessible in the execution cell
global uploaded_article_filenames
global first_uploaded_article_filename

uploaded_article_filenames = []
first_uploaded_article_filename = ""

if not uploaded:
    print("\nNo article file(s) were uploaded.")
else:
    uploaded_article_filenames = list(uploaded.keys())
    print(f"\nUploaded {len(uploaded_article_filenames)} file(s): {', '.join(uploaded_article_filenames)}")

    # Save all uploaded files
    for filename in uploaded_article_filenames:
        destination_path = os.path.join(input_dir, filename)
        with open(destination_path, 'wb') as f:
            f.write(uploaded[filename])
        print(f" - Saved '{filename}' to {input_dir}")

    # Store the name of the *first* file for potential single-article use
    if uploaded_article_filenames:
        first_uploaded_article_filename = uploaded_article_filenames[0]
        print(f"\n(First file detected: '{first_uploaded_article_filename}')")

# Optional: Export for shell access if needed elsewhere, though Python var is preferred
# %env FIRST_FILENAME={first_uploaded_article_filename}

Upload ONE article file for single mode or multi-text:
For multi-text, place one article after the next within same file.
For best results, label each article with a headline (e.g. 'Article 1')
"and make explicit its title & content (e.g. Title 1: 'XXX'. Content 1: XXX)


Saving mysa_article_en.txt to mysa_article_en.txt

Uploaded 1 file(s): mysa_article_en.txt
 - Saved 'mysa_article_en.txt' to /content/input/

(First file detected: 'mysa_article_en.txt')


In [3]:
#@title ▶️ Upload Image File(s) (not needed for Audio only)
import os
import zipfile
from google.colab import files

# Create the destination directory if it doesn't exist
!mkdir -p /content/input/images

# Upload a zip file containing the images
uploaded = files.upload()

for image_filename in uploaded:
    # Construct the destination path within the "input" folder
    destination_path = os.path.join('/content/input/images/', image_filename)

    # Write the uploaded file to the destination path
    with open(destination_path, 'wb') as f:
        f.write(uploaded[image_filename])

    print(f"File '{image_filename}' uploaded to /content/input/images/")

Saving mysa-logo.png to mysa-logo.png
File 'mysa-logo.png' uploaded to /content/input/images/


In [4]:
#@title ▶️ Upload Splash Image File (not needed for Audio only)
#@markdown If you want your video to start with a specific title card or splash image, upload a single image file below. If you upload one, it will be automatically used when you run the execution step. If you skip this, no splash image will be added.
import os
from google.colab import files

# Define the target directory
splash_target_dir = '/content/input/images/'

print("Choose the single splash image file you want to upload:")
uploaded = files.upload()

splash_image_filename = ""

# Check the dictionary returned by files.upload()
if not uploaded:
    print("\nNo splash image was uploaded.")
elif len(uploaded) == 1:
    # Exactly one file uploaded, process it
    splash_filename = list(uploaded.keys())[0]
    splash_content = uploaded[splash_filename]
    destination_path = os.path.join(splash_target_dir, splash_filename)

    # Write the file
    try:
        with open(destination_path, 'wb') as f:
            f.write(splash_content)
        # Store the relative path needed by the execution script
        splash_image_filename = splash_filename
        print(f"\n✅ Splash image '{splash_image_filename}' uploaded successfully.")
    except Exception as e:
        print(f"\n❌ Error saving file '{splash_image_filename}': {e}")

else:
    # More than one file uploaded
    print(f"\n⚠️ Warning: {len(uploaded)} files were selected. Only one splash image is supported.")
    # Process only the first file
    splash_filename = list(uploaded.keys())[0]
    splash_content = uploaded[splash_filename]
    destination_path = os.path.join(splash_target_dir, splash_filename)
    print(f"   Using only the first file: '{splash_filename}'")

    # Write the first file
    try:
        with open(destination_path, 'wb') as f:
            f.write(splash_content)
        # Store the relative path needed by the execution script
        splash_image_filename = splash_filename
        print(f"✅ Splash image '{splash_image_filename}' uploaded successfully.")
    except Exception as e:
        print(f"\n❌ Error saving file '{splash_image_filename}': {e}")

# Make sure the variable is globally accessible if the execution cell needs it directly
# This ensures the variable exists even if no file was uploaded (it will be empty)
global splash_image_filename

Choose the single splash image file you want to upload:



No splash image was uploaded.


You should now be able to find this article/its images/its splash image in Colab's file browser system on the left, if you click on the Folder icon > "input".

# **2. Get the Solution Code From GitHub**

In [5]:
#@title ▶️ Download GitHub Code and Set Up Google Cloud Config
!git clone https://github.com/google-marketing-solutions/article-to-video.git
# This changes the code's default config.yml file to use your Google Cloud Project ID and GCS Bucket Name.
! > /content/article-to-video/config.yml
!echo "gcp_project: \"$PROJECT_ID\"" >> /content/article-to-video/config.yml
!echo 'gcp_location: "us-central1"' >> /content/article-to-video/config.yml
!echo 'output_path: "output"' >> /content/article-to-video/config.yml
!echo "gcs_bucket_name: \"$GCS_BUCKET\"" >> /content/article-to-video/config.yml
!echo 'gcs_bucket_text_path: "text"' >> /content/article-to-video/config.yml
!echo 'gcs_bucket_image_path: "images"' >> /content/article-to-video/config.yml

Cloning into 'article-to-video'...
remote: Enumerating objects: 888, done.
remote: Counting objects: 100% (299/299), done.
remote: Compressing objects: 100% (223/223), done.
remote: Total 888 (delta 177), reused 166 (delta 75), pack-reused 589 (from 1)
Receiving objects: 100% (888/888), 220.75 MiB | 27.47 MiB/s, done.
Resolving deltas: 100% (414/414), done.


# **3. Get the Output!**

In [6]:
#@title ▶️ Download Necessary Libraries
%cd article-to-video/
!pip3 install google-cloud-speech google-cloud-texttospeech msgspec srt
!apt-get install imagemagick

import os

# Create a custom policy.xml file
# (You'll need to define your desired policy settings in this file)
policy_content = """
<policymap>
  <!-- Your custom policy settings here -->
  <policy domain="resource" name="memory" value="8GiB"/>
</policymap>
"""

with open("my_policy.xml", "w") as f:
  f.write(policy_content)

# Set the MAGICK_CONFIGURE_PATH environment variable
os.environ["MAGICK_CONFIGURE_PATH"] = "/content/article-to-video"

!cp /content/article-to-video/my_policy.xml /etc/ImageMagick-6/policy.xml
!cat /etc/ImageMagick-6/policy.xml

/content/article-to-video
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 21.5 MB/s eta 0:00:00
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22427 sha256=026b2117390ce589d63aba3d31fcbeda27c530788c12bba042176276883da365
  Stored in directory: /root/.cache/pip/wheels/7e/75/5b/e1d5c3756631e4bda806f6cc9640153b39484bb6f7b0b8def3
Successfully built srt


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts
  imagemagick-6-common imagemagick-6.q16 libdjvulibre-text libdjvulibre21
  libfftw3-double3 libgs9 libgs9-common libidn12 libijs-0.35 libjbig2dec0
  libjxr-tools libjxr0 liblqr-1-0 libmagickcore-6.q16-6
  libmagickcore-6.q16-6-extra libmagickwand-6.q16-6 libnetpbm10
  libwmflite-0.2-7 netpbm poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre
  ghostscript-x imagemagick-doc autotrace cups-bsd | lpr | lprng enscript gimp
  gnuplot grads hp2xx html2ps libwmf-bin mplayer povray radiance sane-utils
  texlive-base-bin transfig ufraw-batch libfftw3-bin libfftw3-dev inkscape
  poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fon

In [7]:
#@title ▶️ Configuration Options for Video Generation
#@markdown ---
#@markdown **Core Settings:**
video_id = "luna4_article_to_video" #@param {type:"string"}
#@markdown *Unique ID for the video. Leave blank for an auto-generated UUID.*

#@markdown ---
#@markdown **Input Handling:**
#@markdown *Article path/files and image directory are determined from the upload steps.*
#@markdown *Splash image path (if any) is determined from the optional splash image upload cell.*
multi_text = False #@param {type:"boolean"}
#@markdown *Check if the script should process multiple article files from the input directory.*

#@markdown ---
#@markdown **Output Features:**
disable_text_overlays = True #@param {type:"boolean"}
#@markdown *Check to disable text overlay generation.*
burn_in_subtitles = True #@param {type:"boolean"}
#@markdown *Check to burn subtitles directly into the video.*
multi_voice = True #@param {type:"boolean"}
#@markdown *Check to enable multi voice (2 narrators). Else it will be 1 narrator*
#@markdown *Language is set in the first setup section.*

#@markdown ---
#@markdown **Execution Control:**
step = "" #@param ["", "audio", "storyboard", "video"]
#@markdown *Optional: Run only a specific step. Leave blank to run all steps.*
debug = True #@param {type:"boolean"}
#@markdown *Check to enable detailed debug logging.*

#@markdown ---

# Print selected options for verification (optional)
print("--- Configuration Settings ---")
print(f"Video ID: {'Auto-generated UUID' if not video_id else video_id}")
print(f"Multi-text Mode: {multi_text}")
# Check if the global variable from the splash upload cell exists and has a value
print(f"Splash Image Path: {'Set by upload cell' if 'splash_image_filename' in globals() and splash_image_filename else 'None'}")
print(f"Disable Text Overlays: {disable_text_overlays}")
print(f"Burn-in Subtitles: {burn_in_subtitles}")
print(f"Multi-Voice: {multi_voice}")
# Language verification could be added here if needed
# try: print(f"Language: {LANGUAGE_OF_ARTICLE}")
# except NameError: print("Language: Not Set (will use default or fail)")
print(f"Specific Step: {'Run All' if not step else step}")
print(f"Debug Logging: {debug}")
print("-----------------------------")

--- Configuration Settings ---
Video ID: luna4_article_to_video
Multi-text Mode: False
Splash Image Path: None
Disable Text Overlays: True
Burn-in Subtitles: True
Multi-Voice: True
Specific Step: Run All
Debug Logging: True
-----------------------------


In [8]:
#@title ▶️ Get Your Output!
import os
import shlex # Used for safer command string construction

# --- 1. Get variables from previous cells ---
try:
    # Check if any article filenames were captured
    if 'uploaded_article_filenames' not in globals() or not uploaded_article_filenames:
       raise NameError("No article filenames were captured from upload.")
    print(f"Found {len(uploaded_article_filenames)} uploaded article file(s).")
except NameError as e:
    print(f"⚠️ ERROR: Could not find article filename variable: {e}")
    print("Please ensure the article upload cell ran successfully.")
    # Force exit or set dummy values if you want it to try and fail
    raise SystemExit("Stopping execution due to missing article files.") from e

try:
    # Defined in the initial setup cell
    language_code = LANGUAGE_OF_ARTICLE
    print(f"Using language: {language_code}")
except NameError:
    print("⚠️ ERROR: LANGUAGE_OF_ARTICLE not set. Please run the language selection cell.")
    language_code = "en-US" # Fallback to default

# --- Get Splash Image Filename (With Extension) ---
splash_image_filename = splash_image_filename if 'splash_image_filename' in globals() else ""
splash_image_arg = "" # Initialize arg string for the command

if splash_image_filename:
    print(f"Found uploaded splash image (filename): {splash_image_filename}")
    splash_image_arg = f"--splash_image={shlex.quote(splash_image_filename)}"
else:
    print("No splash image provided.")

# --- Get Optional variables from form cell ---
# These variables (video_id, multi_text, etc.) exist from running the form cell
print(f"Using Video ID: {'Auto-generated UUID' if not video_id else video_id}")
print(f"Using Multi-text Mode: {multi_text}")
# ... print other form variables if desired ...

# --- Fixed paths ---
image_dir_arg_val = "../input/images/" # Relative path from script execution dir

# --- 2. Construct the command string ---
command_parts = ["python3", "video_generator_execution.py"]

if multi_voice:
    print("Adding --multi_voice flag.")
    command_parts.append("--multi_voice")

# --- Handle Article Path / Multi-Text ---
if multi_text:
    print("Adding --multi_text flag.")
    command_parts.append("--multi_text")
    # **ASSUMPTION:** Script knows where to find multiple files (e.g., '../input/')

# Use the first uploaded filename
if not first_uploaded_article_filename:
    print("⚠️ ERROR: No article filename available for single-article mode.")
    raise SystemExit("Stopping execution.")
article_path_arg_val = f"../input/{first_uploaded_article_filename}"
print(f"Adding --article_path: {article_path_arg_val}")
command_parts.append(f"--article_path={shlex.quote(article_path_arg_val)}")

# Add other required arguments
print(f"Adding --image_dir: {image_dir_arg_val}")
command_parts.append(f"--image_dir={shlex.quote(image_dir_arg_val)}")
print(f"Adding --language: {language_code}")
command_parts.append(f"--language={shlex.quote(language_code)}")

# Add other optional arguments conditionally
if video_id:
    print(f"Adding --video_id: {video_id}")
    command_parts.append(f"--video_id={shlex.quote(video_id)}")

# Add the correctly formatted splash image arg (if it exists)
if splash_image_arg: # Check the generated argument string
    print(f"Adding splash image argument: {splash_image_arg}")
    command_parts.append(splash_image_arg) # Append the pre-formatted/quoted string

if disable_text_overlays:
    print("Adding --disable_text_overlays flag.")
    command_parts.append("--disable_text_overlays")
if burn_in_subtitles:
    print("Adding --burn_in_subtitles flag.")
    command_parts.append("--burn_in_subtitles")
if step:
    print(f"Adding --step: {step}")
    command_parts.append(f"--step={shlex.quote(step)}")
if debug:
    print("Adding --debug flag.")
    command_parts.append("--debug")

# Join parts into the final command string
final_command = " ".join(command_parts)

# --- 3. Execute the command ---
print("\n" + "="*30)
print("Constructed command:")
print(final_command)
print("="*30 + "\n")

print("Navigating to script directory...")
try:
    os.chdir('/content/article-to-video/')
    print(f"Current directory: {os.getcwd()}")

    print("\nExecuting video generation script...")
    # Run the command using the ! magic
    !{final_command}

    print("\n✅ Script execution finished.")

except FileNotFoundError:
    print("\n❌ ERROR: Directory '/content/article-to-video/' not found.")
    print("   Please ensure the 'git clone' step ran successfully.")
except SystemExit as e:
    print(f"\n❌ Execution stopped: {e}")
except Exception as e:
     print(f"\n❌ An error occurred during script execution: {e}")

Found 1 uploaded article file(s).
Using language: en-GB
No splash image provided.
Using Video ID: luna4_article_to_video
Using Multi-text Mode: False
Adding --multi_voice flag.
Adding --article_path: ../input/mysa_article_en.txt
Adding --image_dir: ../input/images/
Adding --language: en-GB
Adding --video_id: luna4_article_to_video
Adding --disable_text_overlays flag.
Adding --burn_in_subtitles flag.
Adding --debug flag.

Constructed command:
python3 video_generator_execution.py --multi_voice --article_path=../input/mysa_article_en.txt --image_dir=../input/images/ --language=en-GB --video_id=luna4_article_to_video --disable_text_overlays --burn_in_subtitles --debug

Navigating to script directory...
Current directory: /content/article-to-video

Executing video generation script...
/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/pyt

🎉 You should now be able to find this video output in Colab's file browser system on the left, if you click on the Folder icon > "content" (if you don't see this, great, go straight to the next one) > "article_to_video" > "your_video_id" (or a random string if you didn't specify one) > "5_withaudiovideo.mp4"

Congrats, you finished! Now you can download the video file and watch it in Google Drive!

e.g. ![Output Folder](https://raw.githubusercontent.com/google-marketing-solutions/article-to-video/main/colab/video_output_folder.png)


(Optional) You can run the cell below to remove the files you uploaded from the /content/uploads directory if you no longer need them in this Colab session.

In [9]:
# @title ▶️ Optional: Clean up /content/ directory
# @markdown Warning: This will delete everything directly inside /content/ EXCEPT the 'sample_data' folder (including the cloned repo, inputs, outputs, etc.).
run_cleanup = False # @param {type:"boolean"}

import os
import glob
import shlex # For safely quoting paths for shell commands

if run_cleanup:
    print("Attempting cleanup of /content/ (excluding sample_data)...")
    items_to_delete = []
    try:
        # List all files and directories directly under /content
        all_content_items = glob.glob('/content/*')

        for item_path in all_content_items:
            item_name = os.path.basename(item_path)
            if item_name == "sample_data":
                print(f" - Skipping '{item_name}' (standard Colab directory)")
            else:
                items_to_delete.append(item_path)

        if not items_to_delete:
            print(" - Nothing found to delete (besides sample_data).")
        else:
            print("\nItems scheduled for deletion:")
            for item in items_to_delete:
                print(f" - {item}")

            print("\nProceeding with deletion...")
            # Loop through the list and delete items
            for item_path in items_to_delete:
                # Quote the path to handle spaces or special characters safely
                quoted_path = shlex.quote(item_path)
                print(f"   Deleting {quoted_path}...")
                !rm -rf {quoted_path}

        print("\nCleanup complete. Refresh the file browser if needed.")

    except Exception as e:
        print(f"\n❌ An error occurred during cleanup: {e}")

else:
    print("Cleanup skipped (checkbox was not checked).")

Cleanup skipped (checkbox was not checked).
